In [3]:
#Import Python packages
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import json
import datetime

# Import Snowflake modules
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

In [4]:
# Get account credentials from a json file
with open("data_scientist_auth.json") as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]
    account = data["account"]

# Specify connection parameters
connection_parameters = {
    "account": account,
    "user": username,
    "password": password,
    "role": "TASTY_BI",
    "warehouse": "TASTY_BI_WH",
    "database": "frostbyte_tasty_bytes",
    "schema": "analytics",
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()

In [5]:
#Function to remove and rename columns after inner join
def remove_dup_join_col(df):
    # capitalise all columns
    for col in df.columns:
        df = df.withColumnRenamed(col, col.upper())
    
    # get list of renamable columns for left and right join
    left_dup_col = [col_name for col_name in df.columns if col_name.startswith('L_')]
    right_dup_col = [col_name for col_name in df.columns if col_name.startswith('R_')]
    old_columns = df.columns
    
    # rename the list with the most number of renamable columns
    # drop the list of columns with less number of renamable columns
    if len(left_dup_col)>len(right_dup_col):
        columns_rename=left_dup_col
        df= df.drop(*right_dup_col)
    else:
        columns_rename=right_dup_col
        df= df.drop(*left_dup_col)
    
    for old_column in old_columns:
        if old_column in columns_rename:
            # get string to remove
            string_to_replace = columns_rename[0][:7]
            # replace starting string
            new_column = old_column.replace(string_to_replace, "")
            df = df.withColumnRenamed(old_column, new_column)
    
    return df

In [6]:
orders_df = session.table("tran_anal")

In [7]:
orders_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"  |"COUNTRY"  |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"  |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_COUNT"  |"SIGN_UP_DATE"  |"BIRTHDAY_DATE"

In [8]:
orders_df = orders_df.filter(F.col("COUNTRY") == "United States")

orders_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"   |"COUNTRY"      |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"  |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_COUNT"  |"SIGN_UP_DATE"  |"BIR

In [9]:
orders_df = orders_df.with_column("YEAR", F.year("ORDER_TS"))
orders_df = orders_df.with_column("MONTH", F.month("ORDER_TS"))
orders_df = orders_df.with_column("YEAR_MONTH", F.concat(F.col("YEAR"), F.col("MONTH")))
orders_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"   |"COUNTRY"      |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"  |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_

In [10]:
#Get last purchase date as a variable
lastdate=orders_df.select(F.max('ORDER_TS')).first()[0]

# get AGE variable
orders_df = orders_df.withColumn("AGE", F.datediff('year', F.col("BIRTHDAY_DATE"),F.lit(lastdate)))
orders_df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"   |"COUNTRY"      |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"  |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"C

In [11]:
churn_YN = orders_df.with_column("CHURN", F.iff(F.col("DTNO") > 9, "1", "0"))

churn_YN.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"         |"COUNTRY"      |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"     |"FAVOURITE_BRA

In [12]:
SUM_SALES_YM_CITY = orders_df.groupBy(F.col("YEAR"),F.col("MONTH"), F.col("CITY")).agg(F.sum('"ORDER_TOTAL"'))
SUM_SALES_YM_CITY.show()

---------------------------------------------------------
|"YEAR"  |"MONTH"  |"CITY"         |"SUM(ORDER_TOTAL)"  |
---------------------------------------------------------
|2021    |10       |San Mateo      |580027.0000         |
|2019    |12       |Denver         |194667.0000         |
|2022    |5        |Denver         |932059.2500         |
|2021    |3        |New York City  |536115.5000         |
|2022    |9        |Seattle        |694222.7500         |
|2022    |10       |San Mateo      |490403.5000         |
|2019    |8        |San Mateo      |52814.0000          |
|2022    |1        |New York City  |966830.2500         |
|2022    |5        |Boston         |834712.2500         |
|2022    |10       |Seattle        |568301.0000         |
---------------------------------------------------------



In [13]:
churn_rate_YM_CITY = churn_YN.group_by(F.col("YEAR"), F.col("MONTH"), F.col("CITY")).agg(F.sum("CHURN")/F.count("CHURN"))

churn_to_sales_CITY = churn_rate_YM_CITY.join(SUM_SALES_YM_CITY, (churn_rate_YM_CITY["YEAR"] == SUM_SALES_YM_CITY["YEAR"]) & (churn_rate_YM_CITY["MONTH"] == SUM_SALES_YM_CITY["MONTH"]) & (churn_rate_YM_CITY["CITY"] == SUM_SALES_YM_CITY["CITY"]), how="inner")
churn_to_sales_CITY = remove_dup_join_col(churn_to_sales_CITY)
churn_to_sales_CITY.show()

----------------------------------------------------------------------------------------------
|"DIVIDE(SUM(CHURN), COUNT(CHURN))"  |"YEAR"  |"MONTH"  |"CITY"         |"SUM(ORDER_TOTAL)"  |
----------------------------------------------------------------------------------------------
|0.5161159125571937                  |2021    |12       |Denver         |833850.5000         |
|0.49650633734156646                 |2022    |8        |New York City  |985278.7500         |
|0.9607376561570494                  |2019    |6        |San Mateo      |32390.0000          |
|0.6807853518641076                  |2021    |7        |San Mateo      |528043.5000         |
|0.7362990903039716                  |2020    |4        |Boston         |240277.0000         |
|0.30618682342820275                 |2022    |10       |Boston         |558852.0000         |
|0.9326036866359447                  |2019    |4        |Denver         |57657.0000          |
|0.6917602579784277                  |2020    |7  

In [14]:
churn_members = churn_YN.group_by(F.col("YEAR"), F.col("MONTH"), F.col("CITY")).agg(F.count_distinct('CUSTOMER_ID'))

churn_to_sales_CITY = churn_to_sales_CITY.join(churn_members, (churn_members["YEAR"] == churn_to_sales_CITY["YEAR"]) & (churn_members["MONTH"] == churn_to_sales_CITY["MONTH"]) & (churn_members["CITY"] == churn_to_sales_CITY["CITY"]), how="inner")

churn_to_sales_new = remove_dup_join_col(churn_to_sales_CITY)
churn_to_sales_new.show()

------------------------------------------------------------------------------------------------------------------------------
|"DIVIDE(SUM(CHURN), COUNT(CHURN))"  |"SUM(ORDER_TOTAL)"  |"YEAR"  |"MONTH"  |"CITY"         |"COUNT(DISTINCT CUSTOMER_ID)"  |
------------------------------------------------------------------------------------------------------------------------------
|0.6762660718971398                  |580027.0000         |2021    |10       |San Mateo      |8204                           |
|0.4690076466316374                  |932059.2500         |2022    |5        |Denver         |8504                           |
|0.6542634580609265                  |536115.5000         |2021    |3        |New York City  |7769                           |
|0.46136521136521136                 |694222.7500         |2022    |9        |Seattle        |6657                           |
|0.8215978334461747                  |194667.0000         |2019    |12       |Denver         |4388             

In [15]:
member_age = churn_YN.group_by(F.col("YEAR"), F.col("MONTH"), F.col("CITY")).agg(F.avg("AGE"))

churn_to_sales_new = churn_to_sales_new.join(member_age, (member_age["YEAR"] == churn_to_sales_new["YEAR"]) & (member_age["MONTH"] == churn_to_sales_new["MONTH"]) & (member_age["CITY"] == churn_to_sales_new["CITY"]), how="inner")

churn_to_sales_new = remove_dup_join_col(churn_to_sales_new)
churn_to_sales_new.show()

-------------------------------------------------------------------------------------------------------------------------------------------
|"DIVIDE(SUM(CHURN), COUNT(CHURN))"  |"SUM(ORDER_TOTAL)"  |"COUNT(DISTINCT CUSTOMER_ID)"  |"YEAR"  |"MONTH"  |"CITY"         |"AVG(AGE)"  |
-------------------------------------------------------------------------------------------------------------------------------------------
|0.8215978334461747                  |194667.0000         |4388                           |2019    |12       |Denver         |49.613406   |
|0.4690076466316374                  |932059.2500         |8504                           |2022    |5        |Denver         |49.883506   |
|0.46136521136521136                 |694222.7500         |6657                           |2022    |9        |Seattle        |49.278067   |
|0.3230442593499918                  |490403.5000         |6546                           |2022    |10       |San Mateo      |49.362486   |
|0.6542634580609265 

In [16]:
pred = churn_to_sales_new.withColumnRenamed("DIVIDE(SUM(CHURN), COUNT(CHURN))","CHURN_RATE")
pred = pred.withColumnRenamed("SUM(ORDER_TOTAL)", "SALES")
pred = pred.withColumnRenamed("COUNT(DISTINCT CUSTOMER_ID)", "DISTINCT_CUSTOMER")
pred = pred.withColumnRenamed("AVG(AGE)", "AVERAGE_AGE")

pred.show()

--------------------------------------------------------------------------------------------------------------
|"CHURN_RATE"         |"SALES"      |"DISTINCT_CUSTOMER"  |"YEAR"  |"MONTH"  |"CITY"         |"AVERAGE_AGE"  |
--------------------------------------------------------------------------------------------------------------
|0.4690076466316374   |932059.2500  |8504                 |2022    |5        |Denver         |49.883506      |
|0.46136521136521136  |694222.7500  |6657                 |2022    |9        |Seattle        |49.278067      |
|0.6762660718971398   |580027.0000  |8204                 |2021    |10       |San Mateo      |49.332590      |
|0.6542634580609265   |536115.5000  |7769                 |2021    |3        |New York City  |49.375713      |
|0.3230442593499918   |490403.5000  |6546                 |2022    |10       |San Mateo      |49.362486      |
|0.8215978334461747   |194667.0000  |4388                 |2019    |12       |Denver         |49.613406      |
|

In [17]:
df = pred.to_pandas()

In [18]:
df

,CHURN_RATE,SALES,DISTINCT_CUSTOMER,YEAR,MONTH,CITY,AVERAGE_AGE
0,0.469008,932059.25,8504,2022,5,Denver,49.883506
1,0.654263,536115.50,7769,2021,3,New York City,49.375713
2,0.676266,580027.00,8204,2021,10,San Mateo,49.332590
3,0.323044,490403.50,6546,2022,10,San Mateo,49.362486
4,0.461365,694222.75,6657,2022,9,Seattle,49.278067
...,...,...,...,...,...,...,...
189,0.597644,701537.00,8629,2022,7,San Mateo,49.200627
190,0.687888,475562.00,6829,2020,12,Boston,49.407567
191,0.836338,172150.00,3256,2019,11,Seattle,48.959494
192,0.761827,315791.00,6092,2020,4,New York City,49.455682


In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [20]:
df = df.drop(axis=1, columns="CITY")
df

,CHURN_RATE,SALES,DISTINCT_CUSTOMER,YEAR,MONTH,AVERAGE_AGE
0,0.469008,932059.25,8504,2022,5,49.883506
1,0.654263,536115.50,7769,2021,3,49.375713
2,0.676266,580027.00,8204,2021,10,49.332590
3,0.323044,490403.50,6546,2022,10,49.362486
4,0.461365,694222.75,6657,2022,9,49.278067
...,...,...,...,...,...,...
189,0.597644,701537.00,8629,2022,7,49.200627
190,0.687888,475562.00,6829,2020,12,49.407567
191,0.836338,172150.00,3256,2019,11,48.959494
192,0.761827,315791.00,6092,2020,4,49.455682


In [24]:
df.loc[df["MONTH"]==10]

,CHURN_RATE,SALES,DISTINCT_CUSTOMER,YEAR,MONTH,AVERAGE_AGE
2,0.676266,580027.00,8204,2021,10,49.332590
3,0.323044,490403.50,6546,2022,10,49.362486
23,0.668934,551812.00,6946,2020,10,49.846388
49,0.824758,279131.50,5258,2020,10,49.326833
52,0.714890,519330.00,7136,2020,10,49.923307
67,0.306187,558852.00,6633,2022,10,49.518890
91,0.925279,58162.50,2531,2019,10,49.423534
103,0.838225,192566.00,4387,2019,10,49.613311
112,0.265328,828738.25,8257,2022,10,49.616867
114,0.271111,568301.00,5904,2022,10,49.395132


In [91]:
train_x = df.drop(axis=1, columns=["SALES", "YEAR"])
train_y = df["SALES"]

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=10)

In [94]:
from math import sqrt
from sklearn.metrics import mean_squared_error

LR = LinearRegression()
LR.fit(X_train, y_train)

y_fitted = LR.predict(X_train)
print(sqrt(mean_squared_error(y_fitted, y_train)), '(Train RMSE)') # the lower the better
print(LR.score(X_train, y_train), '(Train R^2 Value)') # higher better

69948.93045674295 (Train RMSE)
0.9255714867910557 (Train R^2 Value)


In [95]:
y_fitted2 = LR.predict(X_test)
print(sqrt(mean_squared_error(y_fitted2, y_test)), '(Test RMSE)') # the lower the better
print(LR.score(X_test, y_test), '(Test R^2 Value)') # higher better

93039.09927575946 (Test RMSE)
0.8946651190403448 (Test R^2 Value)


In [98]:
# Coefficients of the logistic regression model
coefficients = LR.coef_
coefficients

array([-7.38158153e+05,  7.36264000e+01,  2.58992191e+03,  3.18397288e+04])

In [126]:
X_train

,CHURN_RATE,DISTINCT_CUSTOMER,MONTH,AVERAGE_AGE
81,0.649929,7332,12,49.447595
43,0.652713,8122,3,49.492447
58,0.731253,6458,6,49.316196
166,0.648108,6903,2,49.766099
180,0.717455,7243,5,49.346016
...,...,...,...,...
113,0.925213,2779,1,48.982644
64,0.697872,6243,8,49.142196
15,0.823013,3356,12,49.054042
125,0.847277,4716,7,49.069062


In [125]:
import joblib

joblib.dump(LR, 'nextMonth.jbl')

OSError: [Errno 22] Invalid argument: 'nextMonth.jbl'